# Analysis of AdaTempScal on ResNet50 for CIFAR10

In [29]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
%autoreload 1

In [31]:
import os
import sys
import time
sys.path.extend(['..'])

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from models import AdaTS, TempScaling, HlogbasedT, DNNbasedT, HistTS, EnsembleTS
%aimport utils
from utils import compare_results, calib_split, get_CIFAR10_C, NumpyDataset, load_model, predict_logits, compute_metrics, onehot_encode
%aimport adats_utils
from adats_utils import fitAdaTS, fitCV_AdaTS, fitHistTS
%aimport mixNmatch_cal
from mixNmatch_cal import ets_calibrate, mir_calibrate

In [32]:
%matplotlib inline

In [33]:
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath} \usepackage{amssymb}')

In [34]:
dev = torch.device('cuda:1')

## Load data and precomputed logits

In [35]:
CIFAR10_PATH = '../../data/CIFAR10'
CIFAR10C_PATH = '../../data/CIFAR-10-C'
MODEL_PATH = '../../trained_models/CIFAR10/resnet50'

In [36]:
X_train = np.load(os.path.join(CIFAR10_PATH, 'train_imas.npy'))
y_train = np.load(os.path.join(CIFAR10_PATH, 'train_labels.npy'))

X_val = np.load(os.path.join(CIFAR10_PATH, 'val_imas.npy'))
y_val = np.load(os.path.join(CIFAR10_PATH, 'val_labels.npy'))

X_test = np.load(os.path.join(CIFAR10_PATH, 'test_imas.npy'))
y_test = np.load(os.path.join(CIFAR10_PATH, 'test_labels.npy'))

In [37]:
Z_train = np.load(os.path.join(MODEL_PATH, 'train_logits.npy'))
Z_val = np.load(os.path.join(MODEL_PATH, 'val_logits.npy'))
Z_test = np.load(os.path.join(MODEL_PATH, 'test_logits.npy'))

### Calibrate models

In [45]:
ix_rand = np.random.permutation(X_val.shape[0])[:1000]

In [46]:
N, dim = Z_train.shape

### Temp-Scal as baseline:
tempScaler = TempScaling()
tempScaler.fit(Z_val[ix_rand], y_val[ix_rand], v=True);

In [47]:
lhaTempScaler = AdaTS(HlogbasedT(dim))
lhaTempScaler = fitAdaTS(lhaTempScaler, Z_val[ix_rand], y_val[ix_rand], epochs=10000, batch_size=1000, lr=1e-4, v=True)

In [48]:
dnnaTempScaler = AdaTS(DNNbasedT(dim))
dnnaTempScaler = fitAdaTS(dnnaTempScaler, Z_val[ix_rand], y_val[ix_rand], epochs=30000, batch_size=1000, lr=1e-4, v=True, weight_decay=1e-2, dev=dev)

In [49]:
PTS = AdaTS(DNNbasedT(dim, hs=[5, 5]))
PTS = fitAdaTS(PTS, Z_val[ix_rand], y_val[ix_rand], epochs=30000, batch_size=1000, lr=1e-4, v=True, weight_decay=1e-2, dev=dev)

In [50]:
hisTS = HistTS()
hisTS.fit(Z_val[ix_rand], y_val[ix_rand])

In [51]:
print('##### Results on train set:')
compare_results(predictions={'Uncal': softmax(Z_train, axis=1),
                             'TempScal': tempScaler.predictive(Z_train),
                             'ETS': ets_calibrate(Z_val[ix_rand], onehot_encode(y_val[ix_rand]), Z_train, dim),
                             'MIR': mir_calibrate(Z_val[ix_rand], onehot_encode(y_val[ix_rand]), Z_train),
                             'logH-TempScal': lhaTempScaler.predictive(Z_train),
                             'DNN-TempScal': dnnaTempScaler.predictive(Z_train),
                             'PTS': PTS.predictive(Z_train),
                             'HistH-TempScal': hisTS.predictive(Z_train)}, target=y_train, from_logits=False);

##### Results on train set:
     Calibrator      Accuracy           ECE           MCE   Brier Score           NLL
Uncal                   99.94%         0.11%        45.80%    1.104e-03     2.701e-03
TempScal                99.94%         5.40%        65.28%    8.687e-03     5.858e-02
ETS                     99.94%         7.43%        43.91%    1.300e-02     8.096e-02
MIR                     99.93%         5.19%        57.97%    1.294e-02     5.899e-02
logH-TempScal           99.94%         5.51%        35.74%    5.226e-03     5.824e-02
DNN-TempScal            99.94%         4.90%        69.01%    1.031e-02     5.482e-02
PTS                     99.94%         5.23%        70.69%    1.041e-02     5.795e-02
HistH-TempScal          99.94%         4.82%        73.47%    1.114e-02     5.430e-02


In [52]:
print('##### Results on validation set:')
compare_results(predictions={'Uncal': softmax(Z_val, axis=1),
                             'TempScal': tempScaler.predictive(Z_val),
                             'ETS': ets_calibrate(Z_val[ix_rand], onehot_encode(y_val[ix_rand]), Z_val, dim),
                             'MIR': mir_calibrate(Z_val[ix_rand], onehot_encode(y_val[ix_rand]), Z_val),
                             'logH-TempScal': lhaTempScaler.predictive(Z_val),
                             'DNN-TempScal': dnnaTempScaler.predictive(Z_val),
                             'PTS': PTS.predictive(Z_val),
                             'HistH-TempScal': hisTS.predictive(Z_val)}, target=y_val, M=15, from_logits=False);

##### Results on validation set:
     Calibrator      Accuracy           ECE           MCE   Brier Score           NLL
Uncal                   86.72%        10.30%        73.80%    2.300e-01     7.737e-01
TempScal                86.72%         2.45%        55.53%    1.967e-01     4.343e-01
ETS                     86.72%         2.81%        80.23%    1.958e-01     4.384e-01
MIR                     86.84%         0.73%        40.96%    1.966e-01     4.271e-01
logH-TempScal           86.72%         5.77%        75.25%    2.125e-01     5.007e-01
DNN-TempScal            86.72%         2.32%         8.67%    1.991e-01     4.545e-01
PTS                     86.72%         2.11%        15.44%    1.974e-01     4.450e-01
HistH-TempScal          86.72%         1.23%         7.75%    1.952e-01     4.420e-01


In [53]:
print('##### Results on test set:')
compare_results(predictions={'Uncal': softmax(Z_test, axis=1),
                             'TempScal': tempScaler.predictive(Z_test),
                             'ETS': ets_calibrate(Z_val[ix_rand], onehot_encode(y_val[ix_rand]), Z_test, dim),
                             'MIR': mir_calibrate(Z_val[ix_rand], onehot_encode(y_val[ix_rand]), Z_test),
                             'logH-TempScal': lhaTempScaler.predictive(Z_test),
                             'DNN-TempScal': dnnaTempScaler.predictive(Z_test),
                             'PTS': PTS.predictive(Z_test),
                             'HistH-TempScal': hisTS.predictive(Z_test)}, target=y_test, M=15, from_logits=False);

##### Results on test set:
     Calibrator      Accuracy           ECE           MCE   Brier Score           NLL
Uncal                   86.13%        10.69%        35.80%    2.392e-01     7.897e-01
TempScal                86.13%         2.36%        16.96%    2.036e-01     4.473e-01
ETS                     86.13%         2.33%        16.63%    2.029e-01     4.517e-01
MIR                     86.15%         0.93%        41.82%    2.053e-01     4.472e-01
logH-TempScal           86.13%         6.12%        28.29%    2.202e-01     5.101e-01
DNN-TempScal            86.13%         2.59%        20.53%    2.074e-01     4.751e-01
PTS                     86.13%         1.96%        25.04%    2.049e-01     4.537e-01
HistH-TempScal          86.13%         1.01%        25.77%    2.022e-01     4.542e-01


In [18]:
mir_calibrate(Z_val, onehot_encode(y_val), Z_test)

array([[4.2555027e-04, 4.2628191e-04, 3.3185841e-03, ..., 3.4811085e-03,
        4.0681465e-04, 4.0351352e-04],
       [5.0656661e-02, 5.0656661e-02, 3.5964174e-03, ..., 8.8662712e-04,
        8.5000002e-01, 2.7230300e-03],
       [1.1013215e-02, 1.1656707e-03, 4.4779759e-04, ..., 4.3475130e-04,
        9.7146118e-01, 5.7745418e-03],
       ...,
       [4.6953425e-04, 4.8792403e-04, 5.5009578e-03, ..., 1.0800964e-03,
        7.6941797e-04, 6.0167187e-04],
       [5.4716974e-01, 2.1985815e-01, 3.2588456e-02, ..., 2.3394532e-03,
        3.3185841e-03, 4.4048734e-02],
       [3.3185841e-03, 4.4335419e-04, 5.7745418e-03, ..., 9.7146118e-01,
        1.5306610e-03, 3.3185841e-03]], dtype=float32)

In [20]:
softmax(Z_test, axis=1)

array([[8.0259728e-09, 8.2769063e-09, 1.2163748e-06, ..., 2.0666739e-06,
        1.6006267e-09, 4.6851145e-10],
       [4.5414656e-04, 5.7007029e-04, 2.1136648e-06, ..., 1.6615165e-07,
        9.9897248e-01, 7.9594372e-07],
       [3.3712506e-05, 2.6184932e-07, 1.5655663e-08, ..., 1.1181473e-08,
        9.9996275e-01, 3.1649208e-06],
       ...,
       [2.3110225e-08, 2.9416979e-08, 2.8897869e-06, ..., 2.3250170e-07,
        1.2595495e-07, 6.8426665e-08],
       [9.1244704e-01, 8.6053014e-02, 7.4484320e-05, ..., 6.6439623e-07,
        1.0405089e-06, 2.0207855e-04],
       [1.6639610e-06, 1.4131823e-08, 7.9446681e-06, ..., 9.9994475e-01,
        3.8702217e-07, 1.3113222e-06]], dtype=float32)

### Temperature distribution

In [1]:
fig, ax = plt.subplots(3, 3, figsize=(25, 25))

Ts = lhaTempScaler.get_T(Z_train)
ax[0, 0].hist(Ts, bins=50)
ax[0, 0].set_xlabel('T', fontsize=22)
ax[0, 0].set_ylabel('Count', fontsize=22)
ax[0, 0].set_title('Train set', fontsize=26)

Ts = lhaTempScaler.get_T(Z_val)
ax[0, 1].hist(Ts, bins=50)
ax[0, 1].set_xlabel('T', fontsize=22)
ax[0, 1].set_ylabel('Count', fontsize=22)
ax[0, 1].set_title('Validation set', fontsize=26)

Ts = lhaTempScaler.get_T(Z_test)
ax[0, 2].hist(Ts, bins=50)
ax[0, 2].set_xlabel('T', fontsize=22)
ax[0, 2].set_ylabel('Count', fontsize=22)
ax[0, 2].set_title('Test set', fontsize=26)



Ts = dnnaTempScaler.get_T(Z_train)
ax[1, 0].hist(Ts, bins=50)
ax[1, 0].set_xlabel('T', fontsize=22)
ax[1, 0].set_ylabel('Count', fontsize=22)
ax[1, 0].set_title('Train set', fontsize=26)

Ts = dnnaTempScaler.get_T(Z_val)
ax[1, 1].hist(Ts, bins=50)
ax[1, 1].set_xlabel('T', fontsize=22)
ax[1, 1].set_ylabel('Count', fontsize=22)
ax[1, 1].set_title('Validation set', fontsize=26)

Ts = dnnaTempScaler.get_T(Z_test)
ax[1, 2].hist(Ts, bins=50)
ax[1, 2].set_xlabel('T', fontsize=22)
ax[1, 2].set_ylabel('Count', fontsize=22)
ax[1, 2].set_title('Test set', fontsize=26)



Ts = bdnnaTempScaler.get_T(Z_train)
ax[2, 0].hist(Ts, bins=50)
ax[2, 0].set_xlabel('T', fontsize=22)
ax[2, 0].set_ylabel('Count', fontsize=22)
ax[2, 0].set_title('Train set', fontsize=26)

Ts = bdnnaTempScaler.get_T(Z_val)
ax[2, 1].hist(Ts, bins=50)
ax[2, 1].set_xlabel('T', fontsize=22)
ax[2, 1].set_ylabel('Count', fontsize=22)
ax[2, 1].set_title('Validation set', fontsize=26)

Ts = bdnnaTempScaler.get_T(Z_test)
ax[2, 2].hist(Ts, bins=50)
ax[2, 2].set_xlabel('T', fontsize=22)
ax[2, 2].set_ylabel('Count', fontsize=22)
ax[2, 2].set_title('Test set', fontsize=26)

for _ax in ax.flatten():
    _ax.tick_params(axis='both', labelsize=18)
    

plt.show()

NameError: name 'plt' is not defined

### Selected temperature for different confidences in test set

In [ ]:
hc, lc, hi, li = calib_split(Z_test, y_test)

ts_aux = TempScaling()
ts_aux.fit(Z_test[hc | hi], y_test[hc | hi]);
T_hc = ts_aux.T.detach().numpy()

ts_aux = TempScaling()
ts_aux.fit(Z_test[lc | li], y_test[lc | li]);
T_lc = ts_aux.T.detach().numpy()


In [ ]:
TslH = lhaTempScaler.get_T(Z_test)
TsDNN = dnnaTempScaler.get_T(Z_test)
TsBDNN = bdnnaTempScaler.get_T(Z_test)

fig, ax = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(23, 25))


ax[0, 0].hist(TslH[hc | hi])
ax[0, 0].axvline(T_hc, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_hc[0]))
ax[0, 0].set_title('High Confidence samples.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TslH[hc | hi])), fontsize=26)
ax[0, 0].legend(fontsize=18)

ax[0, 1].hist(TslH[lc | li])
ax[0, 1].axvline(T_lc, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_lc[0]))
ax[0, 1].set_title('Low Confidence samples.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TslH[lc | li])), fontsize=26)
ax[0, 1].legend(fontsize=18)



ax[1, 0].hist(TsDNN[hc | hi])
ax[1, 0].axvline(T_hc, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_hc[0]))
ax[1, 0].set_title('High Confidence samples.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsDNN[hc | hi])), fontsize=26)
ax[1, 0].legend(fontsize=18)

ax[1, 1].hist(TsDNN[lc | li])
ax[1, 1].axvline(T_lc, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_lc[0]))
ax[1, 1].set_title('Low Confidence samples.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsDNN[lc | li])), fontsize=26)
ax[1, 1].legend(fontsize=18)



ax[2, 0].hist(TsBDNN[hc | hi])
ax[2, 0].axvline(T_hc, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_hc[0]))
ax[2, 0].set_title('High Confidence samples.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsBDNN[hc | hi])), fontsize=26)
ax[2, 0].legend(fontsize=18)

ax[2, 1].hist(TsBDNN[lc | li])
ax[2, 1].axvline(T_lc, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_lc[0]))
ax[2, 1].set_title('Low Confidence samples.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsBDNN[lc | li])), fontsize=26)
ax[2, 1].legend(fontsize=18)

for _ax in ax.flatten():
    _ax.yaxis.set_tick_params(labelleft=True)
    _ax.xaxis.set_tick_params(labelbottom=True)
    _ax.set_xlabel('T', fontsize=22)
    _ax.set_ylabel('Count', fontsize=22)
    _ax.tick_params(axis='both', labelsize=18)
    
plt.tight_layout()
plt.show();

### According to quantiles

In [ ]:
test_probs = softmax(Z_test, axis=1)
test_confs = np.max(test_probs, axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(25, 8))

ax.hist(test_confs, bins=200)

ax.set_xlabel('Confidence', fontsize=22)
ax.set_ylabel('Count', fontsize=22)
ax.tick_params(axis='both', labelsize=18)

plt.show();

In [ ]:
ix = np.argsort(test_confs)

q1, q2, q3, q4 = ix[:len(test_confs)//4], ix[len(test_confs)//4:len(test_confs)//2], ix[len(test_confs)//2: 3*len(test_confs)//4], ix[3*len(test_confs)//4:]


ts_aux = TempScaling()
ts_aux.fit(Z_test[q1], y_test[q1]);
T_q1 = ts_aux.T.detach().numpy()

ts_aux = TempScaling()
ts_aux.fit(Z_test[q2], y_test[q2]);
T_q2 = ts_aux.T.detach().numpy()

ts_aux = TempScaling()
ts_aux.fit(Z_test[q3], y_test[q3]);
T_q3 = ts_aux.T.detach().numpy()

ts_aux = TempScaling()
ts_aux.fit(Z_test[q4], y_test[q4]);
T_q4 = ts_aux.T.detach().numpy()



TslH = lhaTempScaler.get_T(Z_test)
TsDNN = dnnaTempScaler.get_T(Z_test)
TsBDNN = bdnnaTempScaler.get_T(Z_test)

In [ ]:
fig, ax = plt.subplots(3, 4, sharex=True, sharey=True, figsize=(30, 25))


ax[0, 0].hist(TslH[q1])
ax[0, 0].axvline(T_q1, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q1[0]))
ax[0, 0].set_title('Samples in Q1.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TslH[q1])), fontsize=26)

ax[0, 1].hist(TslH[q2])
ax[0, 1].axvline(T_q2, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q2[0]))
ax[0, 1].set_title('Samples in Q2.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TslH[q2])), fontsize=26)

ax[0, 2].hist(TslH[q3])
ax[0, 2].axvline(T_q3, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q3[0]))
ax[0, 2].set_title('Samples in Q3.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TslH[q3])), fontsize=26)

ax[0, 3].hist(TslH[q4])
ax[0, 3].axvline(T_q4, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q4[0]))
ax[0, 3].set_title('Samples in Q4.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TslH[q4])), fontsize=26)



ax[1, 0].hist(TsDNN[q1])
ax[1, 0].axvline(T_q1, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q1[0]))
ax[1, 0].set_title('Samples in Q1.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsDNN[q1])), fontsize=26)

ax[1, 1].hist(TsDNN[q2])
ax[1, 1].axvline(T_q2, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q2[0]))
ax[1, 1].set_title('Samples in Q2.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsDNN[q2])), fontsize=26)

ax[1, 2].hist(TsDNN[q3])
ax[1, 2].axvline(T_q3, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q3[0]))
ax[1, 2].set_title('Samples in Q3.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsDNN[q3])), fontsize=26)

ax[1, 3].hist(TsDNN[q4])
ax[1, 3].axvline(T_q4, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q4[0]))
ax[1, 3].set_title('Samples in Q4.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsDNN[q4])), fontsize=26)



ax[2, 0].hist(TsBDNN[q1])
ax[2, 0].axvline(T_q1, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q1[0]))
ax[2, 0].set_title('Samples in Q1.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsBDNN[q1])), fontsize=26)

ax[2, 1].hist(TsBDNN[q2])
ax[2, 1].axvline(T_q2, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q2[0]))
ax[2, 1].set_title('Samples in Q2.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsBDNN[q2])), fontsize=26)

ax[2, 2].hist(TsBDNN[q3])
ax[2, 2].axvline(T_q3, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q3[0]))
ax[2, 2].set_title('Samples in Q3.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsBDNN[q3])), fontsize=26)

ax[2, 3].hist(TsBDNN[q4])
ax[2, 3].axvline(T_q4, ls='--', lw=2, c='red', label='Optimum Temperature $T = {:.3f}$'.format(T_q4[0]))
ax[2, 3].set_title('Samples in Q4.\n $\mathbb{{E}}[T] = {:.3f}$'.format(np.mean(TsBDNN[q4])), fontsize=26)



for _ax in ax.flatten():
    _ax.yaxis.set_tick_params(labelleft=True)
    _ax.xaxis.set_tick_params(labelbottom=True)
    _ax.legend(fontsize=18)
    _ax.set_xlabel('T', fontsize=22)
    _ax.set_ylabel('Count', fontsize=22)
    _ax.tick_params(axis='both', labelsize=18)

plt.show();

## Corruption Robustness: CIFAR10-C

In [ ]:
cifar10c = get_CIFAR10_C(CIFAR10C_PATH)

categories = list(cifar10c.keys())
categories.remove('labels')

In [ ]:
cifar10_transforms_test=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
net = load_model('resnet50', 'cifar10', model_path='../../trained_models') 

### Robustness without exposure to corrupted samples

In [ ]:
cifar10c_logits = {}

for category in categories:
    cifar10c_logits[category] = {}
    print('Computing predictions for corruption: {}'.format(category))
    for severity in range(1, 6):
        test_data = NumpyDataset(cifar10c[category][severity], cifar10c['labels'][:10000], transform=cifar10_transforms_test)
        test_dataloader = DataLoader(test_data, batch_size=256)
        cifar10c_logits[category][severity] = predict_logits(net, test_dataloader, torch.device('cuda'))

In [ ]:
cifar10c_metrics_uncal = {}
cifar10c_metrics_tscal = {}
cifar10c_metrics_lhtscal = {}
cifar10c_metrics_dnntscal = {}
cifar10c_metrics_bdnntscal = {}
cifar10c_metrics_histtscal = {}

for category in categories:
    cifar10c_metrics_uncal[category] = {}
    cifar10c_metrics_tscal[category] = {}
    cifar10c_metrics_lhtscal[category] = {}
    cifar10c_metrics_dnntscal[category] = {}
    cifar10c_metrics_bdnntscal[category] = {}
    cifar10c_metrics_histtscal[category] = {}
    print('Computing metrics for corruption: {}'.format(category))
    for severity in range(1, 6):
        cifar10c_metrics_uncal[category][severity] = compute_metrics(cifar10c_logits[category][severity], cifar10c['labels'][:10000])
        cifar10c_metrics_tscal[category][severity] = compute_metrics(tempScaler.predictive(cifar10c_logits[category][severity]), cifar10c['labels'][:10000], from_logits=False)
        cifar10c_metrics_lhtscal[category][severity] = compute_metrics(lhaTempScaler.predictive(cifar10c_logits[category][severity]), cifar10c['labels'][:10000], from_logits=False)
        cifar10c_metrics_dnntscal[category][severity] = compute_metrics(dnnaTempScaler.predictive(cifar10c_logits[category][severity]), cifar10c['labels'][:10000], from_logits=False)
        cifar10c_metrics_bdnntscal[category][severity] = compute_metrics(bdnnaTempScaler.predictive(cifar10c_logits[category][severity]), cifar10c['labels'][:10000], from_logits=False)
        cifar10c_metrics_histtscal[category][severity] = compute_metrics(hisTS.predictive(cifar10c_logits[category][severity]), cifar10c['labels'][:10000], from_logits=False)

In [ ]:
mean_sev_uncal = np.zeros((5, 4))
mean_sev_tscal = np.zeros((5, 4))
mean_sev_lhtscal = np.zeros((5, 4))
mean_sev_dnntscal = np.zeros((5, 4))
mean_sev_bdnntscal = np.zeros((5, 4))
mean_sev_histtscal = np.zeros((5, 4))


for i in range(5):
    for cat in categories:
        mean_sev_uncal[i] += cifar10c_metrics_uncal[cat][i+1]
        mean_sev_tscal[i] += cifar10c_metrics_tscal[cat][i+1]
        mean_sev_lhtscal[i] += cifar10c_metrics_lhtscal[cat][i+1]
        mean_sev_dnntscal[i] += cifar10c_metrics_dnntscal[cat][i+1]
        mean_sev_bdnntscal[i] += cifar10c_metrics_bdnntscal[cat][i+1]
        mean_sev_histtscal[i] += cifar10c_metrics_histtscal[cat][i+1]
        
mean_sev_uncal /= len(categories)
mean_sev_tscal /= len(categories)
mean_sev_lhtscal /= len(categories)
mean_sev_dnntscal /= len(categories)
mean_sev_bdnntscal /= len(categories)
mean_sev_histtscal /= len(categories)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(28, 33))

# ax[0].plot(mean_sev_uncal[:, 1], ls='--', marker='*', label='Uncalibrated')
ax[0].plot(mean_sev_tscal[:, 1], ls='--', marker='*', label='Temp-Scaling')
ax[0].plot(mean_sev_lhtscal[:, 1], ls='--', marker='*', label='$\log H$ Temp-Scaling')
ax[0].plot(mean_sev_dnntscal[:, 1], ls='--', marker='*', label='DNN Temp-Scaling')
ax[0].plot(mean_sev_bdnntscal[:, 1], ls='--', marker='*', label='Big DNN Temp-Scaling')
ax[0].plot(mean_sev_histtscal[:, 1], ls='--', marker='*', label='H Hist Temp-Scaling')
ax[0].set_ylabel('ECE', fontsize=22)

# ax[1].plot(mnll_sev_uncal, label='Uncalibrated')
ax[1].plot(mean_sev_tscal[:, 3], ls='--', marker='*', label='Temp-Scaling')
ax[1].plot(mean_sev_lhtscal[:, 3], ls='--', marker='*', label='$\log H$ Temp-Scaling')
ax[1].plot(mean_sev_dnntscal[:, 3], ls='--', marker='*', label='DNN Temp-Scaling')
ax[1].plot(mean_sev_bdnntscal[:, 3], ls='--', marker='*', label='Big DNN Temp-Scaling')
ax[1].plot(mean_sev_histtscal[:, 3], ls='--', marker='*', label='H Hist Temp-Scaling')
ax[1].set_ylabel('NLL', fontsize=22)

ax[2].plot(mean_sev_tscal[:, 2], ls='--', marker='*', label='Temp-Scaling')
ax[2].plot(mean_sev_lhtscal[:, 2], ls='--', marker='*', label='$\log H$ Temp-Scaling')
ax[2].plot(mean_sev_dnntscal[:, 2], ls='--', marker='*', label='DNN Temp-Scaling')
ax[2].plot(mean_sev_bdnntscal[:, 2], ls='--', marker='*', label='Big DNN Temp-Scaling')
ax[2].plot(mean_sev_histtscal[:, 2], ls='--', marker='*', label='H Hist Temp-Scaling')
ax[2].set_ylabel('Brier Score', fontsize=22)

for _ax in ax.flatten():
    _ax.set_xticks(np.arange(5))
    _ax.set_xticklabels(np.arange(5) + 1)
    _ax.legend(fontsize=18)
    _ax.tick_params(axis='both', labelsize=18)
    _ax.set_xlabel('Corruption level', fontsize=22)

plt.show()

In [ ]:
cifar10c_TslH= {}
cifar10c_TsDNN= {}

for category in categories:
    cifar10c_TslH[category] = {}
    cifar10c_TsDNN[category] = {}
    print('Computing Ts for corruption: {}'.format(category))
    for severity in range(1, 6):
        cifar10c_TslH[category][severity] = lhaTempScaler.get_T(cifar10c_logits[category][severity])
        cifar10c_TsDNN[category][severity] = dnnaTempScaler.get_T(cifar10c_logits[category][severity])

In [ ]:
mean_TslH = np.zeros(5)
mean_TsDNN = np.zeros(5)
for i in range(5):
    for cat in categories:
        mean_TslH[i] += np.mean(cifar10c_TslH[cat][i+1])
        mean_TsDNN[i] += np.mean(cifar10c_TsDNN[cat][i+1])
        
mean_TslH /= len(categories)
mean_TsDNN /= len(categories)

In [ ]:
print(mean_TslH)
print(mean_TsDNN)

### Exposure to corruptions

In [ ]:
rnd_idx = np.random.permutation(10000)
idx_train = rnd_idx[:3000]
idx_test = rnd_idx[3000:]

In [ ]:
train_set = np.vstack([cifar10c_logits[cat][i+1][idx_train] for cat in categories for i in range(5)])
y_train = np.hstack([cifar10c['labels'][idx_train] for cat in categories for i in range(5)])

In [ ]:
### Temp-Scal as baseline:
tempScaler = TempScaling()
tempScaler.fit(train_set, y_train, v=True);

In [ ]:
lhaTempScaler = AdaTS(HlogbasedT(dim))
lhaTempScaler = fitAdaTS(lhaTempScaler, train_set, y_train, epochs=10000, batch_size=1000, lr=1e-4, v=True, dev=dev)

In [ ]:
dnnaTempScaler = AdaTS(DNNbasedT(dim))
dnnaTempScaler = fitAdaTS(dnnaTempScaler, train_set, y_train, epochs=10000, batch_size=1000, lr=1e-4, v=True, weight_decay=1e-2, dev=dev)

In [ ]:
bdnnaTempScaler = AdaTS(DNNbasedT(dim, hs=[2*dim, 2*dim]))
bdnnaTempScaler = fitAdaTS(bdnnaTempScaler, train_set, y_train, epochs=10000, batch_size=1000, lr=1e-4, v=True, weight_decay=1e-2, dev=dev)

In [ ]:
hisTS = HistTS()
hisTS.fit(train_set, y_train)

In [ ]:
cifar10c_metrics_uncal = {}
cifar10c_metrics_tscal = {}
cifar10c_metrics_lhtscal = {}
cifar10c_metrics_dnntscal = {}
cifar10c_metrics_bdnntscal = {}
cifar10c_metrics_histtscal = {}

for category in categories:
    cifar10c_metrics_uncal[category] = {}
    cifar10c_metrics_tscal[category] = {}
    cifar10c_metrics_lhtscal[category] = {}
    cifar10c_metrics_dnntscal[category] = {}
    cifar10c_metrics_bdnntscal[category] = {}
    cifar10c_metrics_histtscal[category] = {}
    print('Computing metrics for corruption: {}'.format(category))
    for severity in range(1, 6):
        cifar10c_metrics_uncal[category][severity] = compute_metrics(cifar10c_logits[category][severity][idx_test], cifar10c['labels'][idx_test])
        cifar10c_metrics_tscal[category][severity] = compute_metrics(tempScaler.predictive(cifar10c_logits[category][severity][idx_test]), cifar10c['labels'][idx_test], from_logits=False)
        cifar10c_metrics_lhtscal[category][severity] = compute_metrics(lhaTempScaler.predictive(cifar10c_logits[category][severity][idx_test]), cifar10c['labels'][idx_test], from_logits=False)
        cifar10c_metrics_dnntscal[category][severity] = compute_metrics(dnnaTempScaler.predictive(cifar10c_logits[category][severity][idx_test]), cifar10c['labels'][idx_test], from_logits=False)
        cifar10c_metrics_bdnntscal[category][severity] = compute_metrics(bdnnaTempScaler.predictive(cifar10c_logits[category][severity][idx_test]), cifar10c['labels'][idx_test], from_logits=False)
        cifar10c_metrics_histtscal[category][severity] = compute_metrics(hisTS.predictive(cifar10c_logits[category][severity][idx_test]), cifar10c['labels'][idx_test], from_logits=False)

In [ ]:
mean_sev_uncal = np.zeros((5, 4))
mean_sev_tscal = np.zeros((5, 4))
mean_sev_lhtscal = np.zeros((5, 4))
mean_sev_dnntscal = np.zeros((5, 4))
mean_sev_bdnntscal = np.zeros((5, 4))
mean_sev_histtscal = np.zeros((5, 4))


for i in range(5):
    for cat in categories:
        mean_sev_uncal[i] += cifar10c_metrics_uncal[cat][i+1]
        mean_sev_tscal[i] += cifar10c_metrics_tscal[cat][i+1]
        mean_sev_lhtscal[i] += cifar10c_metrics_lhtscal[cat][i+1]
        mean_sev_dnntscal[i] += cifar10c_metrics_dnntscal[cat][i+1]
        mean_sev_bdnntscal[i] += cifar10c_metrics_bdnntscal[cat][i+1]
        mean_sev_histtscal[i] += cifar10c_metrics_histtscal[cat][i+1]
        
mean_sev_uncal /= len(categories)
mean_sev_tscal /= len(categories)
mean_sev_lhtscal /= len(categories)
mean_sev_dnntscal /= len(categories)
mean_sev_bdnntscal /= len(categories)
mean_sev_histtscal /= len(categories)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(28, 33))

# ax[0].plot(mean_sev_uncal[:, 1], ls='--', marker='*', label='Uncalibrated')
ax[0].plot(mean_sev_tscal[:, 1], ls='--', marker='*', label='Temp-Scaling')
ax[0].plot(mean_sev_lhtscal[:, 1], ls='--', marker='*', label='$\log H$ Temp-Scaling')
ax[0].plot(mean_sev_dnntscal[:, 1], ls='--', marker='*', label='DNN Temp-Scaling')
ax[0].plot(mean_sev_bdnntscal[:, 1], ls='--', marker='*', label='Big DNN Temp-Scaling')
ax[0].plot(mean_sev_histtscal[:, 1], ls='--', marker='*', label='H Hist Temp-Scaling')
ax[0].set_ylabel('ECE', fontsize=22)

# ax[1].plot(mnll_sev_uncal, label='Uncalibrated')
ax[1].plot(mean_sev_tscal[:, 3], ls='--', marker='*', label='Temp-Scaling')
ax[1].plot(mean_sev_lhtscal[:, 3], ls='--', marker='*', label='$\log H$ Temp-Scaling')
ax[1].plot(mean_sev_dnntscal[:, 3], ls='--', marker='*', label='DNN Temp-Scaling')
ax[1].plot(mean_sev_bdnntscal[:, 3], ls='--', marker='*', label='Big DNN Temp-Scaling')
ax[1].plot(mean_sev_histtscal[:, 3], ls='--', marker='*', label='H Hist Temp-Scaling')
ax[1].set_ylabel('NLL', fontsize=22)

ax[2].plot(mean_sev_tscal[:, 2], ls='--', marker='*', label='Temp-Scaling')
ax[2].plot(mean_sev_lhtscal[:, 2], ls='--', marker='*', label='$\log H$ Temp-Scaling')
ax[2].plot(mean_sev_dnntscal[:, 2], ls='--', marker='*', label='DNN Temp-Scaling')
ax[2].plot(mean_sev_bdnntscal[:, 2], ls='--', marker='*', label='Big DNN Temp-Scaling')
ax[2].plot(mean_sev_histtscal[:, 2], ls='--', marker='*', label='H Hist Temp-Scaling')
ax[2].set_ylabel('Brier Score', fontsize=22)

for _ax in ax.flatten():
    _ax.set_xticks(np.arange(5))
    _ax.set_xticklabels(np.arange(5) + 1)
    _ax.legend(fontsize=18)
    _ax.tick_params(axis='both', labelsize=18)
    _ax.set_xlabel('Corruption level', fontsize=22)

plt.show()